<a href="https://colab.research.google.com/github/BartBachrach/Yellow_Project/blob/Database/movie_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 88.7 kB in 3s (32.9 kB/s)
Reading package lists... Done


In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-15 20:54:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.90MB/s    in 0.2s    

2022-08-15 20:54:06 (4.90 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Movie-Recs").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://yellow-movie-data.s3.amazonaws.com/sample_table2.csv"
spark.sparkContext.addFile(url)
sample_data_df = spark.read.csv(SparkFiles.get("sample_table2.csv"), sep=",", header=True, inferSchema=True)

In [8]:
# Show DataFrame
sample_data_df.show(5)

+----+----+------------+-------+-----+---------+-----------+----------+--------------------+--------------------+-------+-------+-----------------+------------------+---------+---------+--------+------+-------+-------+-----+------+-----+--------+------+-------+------+----+-----------+---+-------+-------+---------+------------------+-----+
| _c0|year|Unnamed: 0.1|movieId|tagId|relevance|        tag|Unnamed: 0|               title|              genres| imdbId| tmdbId|number of ratings|            rating|Adventure|Animation|Children|Comedy|Fantasy|Romance|Drama|Action|Crime|Thriller|Horror|Mystery|Sci-Fi|IMAX|Documentary|War|Musical|Western|Film-Noir|(no genres listed)|class|
+----+----+------------+-------+-----+---------+-----------+----------+--------------------+--------------------+-------+-------+-----------------+------------------+---------+---------+--------+------+-------+-------+-----+------+-----+--------+------+-------+------+----+-----------+---+-------+-------+---------+---

In [9]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://yellow-movie-data.s3.amazonaws.com/website_df.csv"
spark.sparkContext.addFile(url)
full_data_df = spark.read.csv(SparkFiles.get("website_df.csv"), sep=",", header=True, inferSchema=True)

In [10]:
full_data_df.show(5)

+---+----+------------+-------+-----+---------+---------+----------+--------------------+--------------------+------+-------+-----------------+------------------+---------+---------+--------+------+-------+-------+-----+------+-----+--------+------+-------+------+----+-----------+---+-------+-------+---------+------------------+-----+
|_c0|year|Unnamed: 0.1|movieId|tagId|relevance|      tag|Unnamed: 0|               title|              genres|imdbId| tmdbId|number of ratings|            rating|Adventure|Animation|Children|Comedy|Fantasy|Romance|Drama|Action|Crime|Thriller|Horror|Mystery|Sci-Fi|IMAX|Documentary|War|Musical|Western|Film-Noir|(no genres listed)|class|
+---+----+------------+-------+-----+---------+---------+----------+--------------------+--------------------+------+-------+-----------------+------------------+---------+---------+--------+------+-------+-------+-----+------+-----+--------+------+-------+------+----+-----------+---+-------+-------+---------+---------------

In [11]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://movie-project.cc10tszik781.us-east-1.rds.amazonaws.com:5432/movies"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
sample_data_df.write.jdbc(url=jdbc_url, table='sample_table', mode=mode, properties=config)

In [12]:
# Write review_id_df to table in RDS
full_data_df.write.jdbc(url=jdbc_url, table='full_table', mode=mode, properties=config)